<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Definições-Iniciais" data-toc-modified-id="Definições-Iniciais-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Definições Iniciais</a></span></li><li><span><a href="#Funções-de-Configuração" data-toc-modified-id="Funções-de-Configuração-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Funções de Configuração</a></span></li><li><span><a href="#Envio-de-E-mails-com-Anexo" data-toc-modified-id="Envio-de-E-mails-com-Anexo-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Envio de E-mails com Anexo</a></span></li><li><span><a href="#Envio-de-E-mails-com-DataFrame-no-Body" data-toc-modified-id="Envio-de-E-mails-com-DataFrame-no-Body-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Envio de E-mails com DataFrame no Body</a></span></li><li><span><a href="#Referências" data-toc-modified-id="Referências-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Referências</a></span></li><li><span><a href="#Dependências" data-toc-modified-id="Dependências-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Dependências</a></span></li><li><span><a href="#Teste-Encapsulado" data-toc-modified-id="Teste-Encapsulado-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Teste Encapsulado</a></span></li></ul></div>

Este notebook tem como objetivo realizar testes com a biblioteca `exchangelib` do Python para o envio de e-mails utilizando o Microsoft Exchange. A partir dela, pretende-se realizar configurações de envio de e-mails simples e, de forma incremental, analisar possibilidades de formatações adicionais nos objetos enviados.

# Definições Iniciais

In [ ]:
# Importando bibliotecas
from dotenv import load_dotenv
import os
from exchangelib import Credentials, Account, Configuration, Message, DELEGATE, \
                        FileAttachment, HTMLBody
import pandas as pd
import io
from pretty_html_table import build_table

In [ ]:
# Lendo variáveis de ambiente
load_dotenv()

# Definindo variáveis de configuração do e-mail
USERNAME = 'thipanini94@gmail.com'
PWD = os.getenv('PWD')
SERVER = 'outlook.office365.com'
MAIL_BOX = 'thipanini94@gmail.com'
MAIL_TO = ['thipanini94@gmail.com']

# Definindo variáveis de formatação do e-mail
MAIL_BODY = "Relatório enviado com sucesso! Dados:<br/><br/>"
DF_ON_BODY = True

# Variáveis testes de anexo de arquivos
CSV_FILEPATH = r'D:\Users\thiagoPanini\data-science-repos\kaggle\01_german_credit_risk\german_credit_data.csv'
PNG_FILEPATH = r'D:\Users\thiagoPanini\data-science-repos\kaggle\01_german_credit_risk\confusion_matrix_final.png'
TXT_FILEPATH = r'D:\Users\thiagoPanini\data-science-repos\kaggle\04_brazilian_e-commerce\dataset_test.txt'
XLSX_FILEPATH = r'C:\Users\thipa\Desktop\certificacao_alteryx\core\praticas\4701229_3BJP240W.xlsx'

FLAG_ATTACHMENT = True
FILE_PATHS = [CSV_FILEPATH, XLSX_FILEPATH, TXT_FILEPATH]

# Funções de Configuração

In [ ]:
# Definindo função de conexão com o servidor
def connect_exchange(username, password, server, mail_box, auto_discover=False, access_type=DELEGATE):
    """
    Realiza a conexão com o servidor através da conta exchange
    
    Parâmetros
    ----------
    :param username: e-mail do usuário com acesso de envio de e-mail do endereço smtp [type: string]
    :param password: senha do usuário com acesso de envio de e-mail do endereço smtp [type: string]
    :param server: servidor responsável por gerenciar o transporte [type: string]
    :param mail_box: endereço primário associado a conta de envio [type: string]
    :param auto_discover: flag para apontar ao EWS utilizando protocolo específico [type: bool, default=False]
    :param access_type: tipo de acesso relacionado as credenciais [type: obj, default=DELEGATE]
    
    Retorno
    -------
    :return account: objeto contendo informações compiladas de uma conta de usuário
    """
    
    # Definindo e configurando credenciais
    creds = Credentials(username=username, password=password)
    config = Configuration(server=server, credentials=creds)
    account = Account(primary_smtp_address=mail_box, credentials=creds,
                      autodiscover=auto_discover, access_type=access_type, config=config)
    
    return account

# Criando função para envio de arquivos anexo
def attach_file(name, df):
    """
    Armazena DataFrames em buffers e transforma o conteúdo em bytes para envio em anexo
    
    Parâmetros
    ----------
    :param name: nome/referência do arquivo com a extensão [type: string]
    :param df: base de dados em formato DataFrame [type: pd.DataFrame]
    
    Retorno
    -------
    :return attachment_list: lista contendo o nome e o conteúdo em bytes do arquivo anexo [type: list]
    """
    
    # Criando buffer para armazenamento de bytes
    buffer = io.BytesIO()
    
    # Resgatando extensão do arquivo
    file_name, file_ext = os.path.splitext(name)
    
    # Salvando arquivo no buffer de acordo com extensão
    if file_ext in ['.csv', '.txt']:
        df.to_csv(buffer)
    elif file_ext == '.xlsx':
        df.to_excel(buffer)
    else:
        print('Extensão inválida. Opções: "csv", "txt" e "xlsx"')
        
    # Lendo buffer e retornando referências do anexo
    buffer_content = buffer.getvalue()
    
    return [name, buffer_content]

# Função para formatação de código html em corpo de e-mail
def format_mail_body(string_mail_body, mail_signature='', **kwargs):
    """
    Função desenvolvida para formatação do corpo do e-mail em formato HTML.
    Opcionalmente, é possível enviar DataFrames em formato de tabela pré formatada
    utilizando a biblioteca pretty_html_table.
    
    Parâmetros
    ----------
    :param string_mail_body: corpo de e-mail em formato de string [type: string]
        *pode conter código html para devida formatação e conversão
    :param **kwargs: argumentos adicionais da função para formatação do corpo
        :arg df: base de dados a ser enviada no corpo [type: pd.DataFrame]
        :arg color: configuração de cor disponibilizada pelo formatador [type: string, default='blue_light']
        :arg font_size: tamanho da fonte [type: string, default='medium']
        :arg font_family: tipo da fonte [type: string, default='Century Gothic']
        :arg text_align: alinhamento do texto [type: string, default='left']
        
    Retorno
    -------
    :return HTMLBody(string): corpo de e-mail devidamente formatado [type: string]
    """
    
    # Extraindo parâmetros
    df = kwargs['df'] if 'df' in kwargs else None
    color = kwargs['color'] if 'color' in kwargs else 'blue_light'
    font_size = kwargs['font_size'] if 'font_size' in kwargs else 'medium'
    font_family = kwargs['font_family'] if 'font_family' in kwargs else 'Century Gothic'
    text_align = kwargs['text_align'] if 'text_align' in kwargs else 'left'
    
    if df is not None:
        html_df = build_table(df, 
                              color=color, 
                              font_size=font_size, 
                              font_family=font_family, 
                              text_align=text_align)
        
        return HTMLBody(string_mail_body + html_df + mail_signature)
    else:
        return HTMLBody(string_mail_body)

# Envio de E-mails com Anexo

In [ ]:
# Configurando conta de envio
account = connect_exchange(username=USERNAME, password=PWD, server=SERVER, mail_box=MAIL_BOX)

# Preparando mensagem
m = Message(account=account,
            subject='Envio de e-mail com anexo',
            body='Teste',
            to_recipients=MAIL_TO)

# Verificando presença de anexos no e-mail
if FLAG_ATTACHMENT: 
    # Lendo bases de diferentes formatos para serem enviadas por email
    df_csv = pd.read_csv(CSV_FILEPATH)
    df_xlsx = pd.read_excel(XLSX_FILEPATH)
    df_txt = pd.read_csv(TXT_FILEPATH)

    # Construindo referência de entradas e bases
    file_names = ['german_credit.csv', 'ayx_core.xlsx', 'brazilian_test.txt']
    file_dfs = [df_csv, df_xlsx, df_txt]

    # Gerando dicionário de entradas e referências
    attach_dict = {file_names.index(name) + 1: {'name': name, 'df': df} for name, df in zip(file_names, file_dfs)}
    attachments = [attach_file(inner_dict['name'], inner_dict['df']) for idx, inner_dict in attach_dict.items()]
    
    # Enviando anexo
    for name, content in attachments or []:
        file = FileAttachment(name=name, content=content)
        m.attach(file)

# Enviando mensagem
m.send_and_save()

# Envio de E-mails com DataFrame no Body

In [ ]:
# Configurando conta de envio
account = connect_exchange(username=USERNAME, password=PWD, server=SERVER, mail_box=MAIL_BOX)

# Verificando envio de bases de dados no corpo do e-mail
if DF_ON_BODY:
    df_xlsx = pd.read_excel(XLSX_FILEPATH)
    html_body = format_mail_body(MAIL_BODY, df=df_xlsx)
else:
    html_body = format_mail_body(MAIL_BODY)
    
# Preparando mensagem
m = Message(account=account,
            subject='Envio de e-mail com DataFrame no body',
            body=html_body,
            to_recipients=MAIL_TO)

# Verificando presença de anexos no e-mail
if FLAG_ATTACHMENT: 
    # Lendo bases de diferentes formatos para serem enviadas por email
    df_csv = pd.read_csv(CSV_FILEPATH)
    df_xlsx = pd.read_excel(XLSX_FILEPATH)
    df_txt = pd.read_csv(TXT_FILEPATH)
    
    # Construindo referência de entradas e bases
    file_names = ['german_credit.csv', 'ayx_core.xlsx', 'brazilian_test.txt']
    file_dfs = [df_csv, df_xlsx, df_txt]

    # Gerando dicionário de entradas e referências
    attach_dict = {file_names.index(name) + 1: {'name': name, 'df': df} for name, df in zip(file_names, file_dfs)}
    attachments = [attach_file(inner_dict['name'], inner_dict['df']) for idx, inner_dict in attach_dict.items()]
    
    # Enviando anexo
    for name, content in attachments or []:
        file = FileAttachment(name=name, content=content)
        m.attach(file)
        
# Enviando mensagem
m.send_and_save()

# Referências

* `pretty_html_table` para formatar DataFrames com código HTML para corpo do email>
    * https://dev.to/siddheshshankar/convert-a-dataframe-into-a-pretty-html-table-and-send-it-over-email-4663
    * https://pypi.org/project/pretty-html-table/
    * https://github.com/sbi-rviot/ph_table
    
    
* `io` para armazenamento de dados em buffer para envio de DataFrames em anexo
    * https://docs.python.org/3/library/io.html#io.StringIO

# Dependências

* `exchangelib==4.1.0`
* `openpyxl==3.0.7`
* `numpy==1.20.2`
* `pandas==1.2.3`
* `pretty-html-table==0.9.dev0`

___
Testes para criação de metadados de bases considerando mais de uma entrada possível na macro
___

In [ ]:
# Lendo bases de dados
file_paths = [CSV_FILEPATH, XLSX_FILEPATH, TXT_FILEPATH]
dfs = []
for path in file_paths:
    try:
        dfs.append([pd.read_csv(path)])
    except:
        dfs.append([pd.read_excel(path)])

# Preparando listas
inputs = [i for i in range(1, 4)]
names = ['german.csv', 'ayx_ore.xlsx', 'brazilian.txt']
dfs = [df_csv, df_xlsx, df_txt]
flag_body = [0, 1, 0]
flag_attach = [1, 1, 0]

# Criando DataFrame vazio
meta_df = pd.DataFrame({})
meta_df['input'] = inputs
meta_df['name'] = names
meta_df['df'] = dfs
meta_df['flag_body'] = flag_body
meta_df['flag_attach'] = flag_attach

meta_df

In [ ]:
# Configurando conta de envio
account = connect_exchange(username=USERNAME, password=PWD, server=SERVER, mail_box=MAIL_BOX)

# Verificando dados a serem enviados no corpo
meta_df_body = meta_df.query('flag_body == 1')
html_df = pd.DataFrame(meta_df_body['df']).iloc[0, :]['df']
html_body = format_mail_body(MAIL_BODY, df=html_df)

# Preparando mensagem
m = Message(account=account,
            subject='Envio de e-mail com DataFrame no body',
            body=html_body,
            to_recipients=MAIL_TO)

# Verificando anexos e preparando estruturas
meta_df_attach = meta_df.query('flag_attach == 1')
file_names = list(meta_df_attach['name'])
file_dfs = list(meta_df_attach['df'])

attach_dict = {file_names.index(name) + 1: {'name': name, 'df': df} for name, df in zip(file_names, file_dfs)}
attachments = [attach_file(inner_dict['name'], inner_dict['df']) for idx, inner_dict in attach_dict.items()]

# Anexando arquivos
for name, content in attachments or []:
    file = FileAttachment(name=name, content=content)
    m.attach(file)
    
# Enviando mensagem
m.send_and_save()

# Teste Encapsulado

In [1]:
# Importando bibliotecas
from dotenv import load_dotenv
import os
from exchangelib import Credentials, Account, Configuration, Message, DELEGATE, \
                        FileAttachment, HTMLBody
import pandas as pd
import io
from pretty_html_table import build_table

In [2]:
# Definindo função de conexão com o servidor
def connect_exchange(username, password, server, mail_box, auto_discover=False, access_type=DELEGATE):
    """
    Realiza a conexão com o servidor através da conta exchange
    
    Parâmetros
    ----------
    :param username: e-mail do usuário com acesso de envio de e-mail do endereço smtp [type: string]
    :param password: senha do usuário com acesso de envio de e-mail do endereço smtp [type: string]
    :param server: servidor responsável por gerenciar o transporte [type: string]
    :param mail_box: endereço primário associado a conta de envio [type: string]
    :param auto_discover: flag para apontar ao EWS utilizando protocolo específico [type: bool, default=False]
    :param access_type: tipo de acesso relacionado as credenciais [type: obj, default=DELEGATE]
    
    Retorno
    -------
    :return account: objeto contendo informações compiladas de uma conta de usuário
    """
    
    # Definindo e configurando credenciais
    creds = Credentials(username=username, password=password)
    config = Configuration(server=server, credentials=creds)
    account = Account(primary_smtp_address=mail_box, credentials=creds,
                      autodiscover=auto_discover, access_type=access_type, config=config)
    
    return account

# Criando função para envio de arquivos anexo
def attach_file(name, df):
    """
    Armazena DataFrames em buffers e transforma o conteúdo em bytes para envio em anexo
    
    Parâmetros
    ----------
    :param name: nome/referência do arquivo com a extensão [type: string]
    :param df: base de dados em formato DataFrame [type: pd.DataFrame]
    
    Retorno
    -------
    :return attachment_list: lista contendo o nome e o conteúdo em bytes do arquivo anexo [type: list]
    """
    
    # Criando buffer para armazenamento de bytes
    buffer = io.BytesIO()
    
    # Resgatando extensão do arquivo
    file_name, file_ext = os.path.splitext(name)
    
    # Salvando arquivo no buffer de acordo com extensão
    if file_ext in ['.csv', '.txt']:
        df.to_csv(buffer)
    elif file_ext == '.xlsx':
        df.to_excel(buffer)
    else:
        print('Extensão inválida. Opções: "csv", "txt" e "xlsx"')
        
    # Lendo buffer e retornando referências do anexo
    buffer_content = buffer.getvalue()
    
    return [name, buffer_content]

# Função para formatação de código html em corpo de e-mail
def format_mail_body(string_mail_body, mail_signature='', **kwargs):
    """
    Função desenvolvida para formatação do corpo do e-mail em formato HTML.
    Opcionalmente, é possível enviar DataFrames em formato de tabela pré formatada
    utilizando a biblioteca pretty_html_table.
    
    Parâmetros
    ----------
    :param string_mail_body: corpo de e-mail em formato de string [type: string]
        *pode conter código html para devida formatação e conversão
    :param **kwargs: argumentos adicionais da função para formatação do corpo
        :arg df: base de dados a ser enviada no corpo [type: pd.DataFrame]
        :arg color: configuração de cor disponibilizada pelo formatador [type: string, default='blue_light']
        :arg font_size: tamanho da fonte [type: string, default='medium']
        :arg font_family: tipo da fonte [type: string, default='Century Gothic']
        :arg text_align: alinhamento do texto [type: string, default='left']
        
    Retorno
    -------
    :return HTMLBody(string): corpo de e-mail devidamente formatado [type: string]
    """
    
    # Extraindo parâmetros
    df = kwargs['df'] if 'df' in kwargs else None
    color = kwargs['color'] if 'color' in kwargs else 'blue_light'
    font_size = kwargs['font_size'] if 'font_size' in kwargs else 'medium'
    font_family = kwargs['font_family'] if 'font_family' in kwargs else 'Century Gothic'
    text_align = kwargs['text_align'] if 'text_align' in kwargs else 'left'
    
    if df is not None:
        html_df = build_table(df, 
                              color=color, 
                              font_size=font_size, 
                              font_family=font_family, 
                              text_align=text_align)
        
        return HTMLBody(string_mail_body + html_df + mail_signature)
    else:
        return HTMLBody(string_mail_body + mail_signature)
    
# Definindo função capaz de encapsular a mensagem
def send_mail_mult_files(meta_df, username, password, server, mail_box, subject, mail_body, 
                         mail_to, mail_signature=''):
    
    # Configurando conta de envio
    account = connect_exchange(username=username, password=password, server=server, mail_box=mail_box)

    # Verificando dados a serem enviados no corpo
    meta_df_body = meta_df.query('flag_body == 1')
    if len(meta_df_body) > 0:
        html_df = pd.DataFrame(meta_df_body['df']).iloc[0, :]['df']
        html_body = format_mail_body(mail_body, df=html_df, mail_signature=mail_signature)
    else:
        html_body = format_mail_body(mail_body, mail_signature=mail_signature)
    
    # Preparando mensagem
    m = Message(account=account,
                subject=subject,
                body=html_body,
                to_recipients=mail_to)
    
    # Verificando anexos e preparando estruturas
    meta_df_attach = meta_df.query('flag_attach == 1')
    file_names = list(meta_df_attach['name'])
    file_dfs = list(meta_df_attach['df'])

    attach_dict = {file_names.index(name) + 1: {'name': name, 'df': df} for name, df in zip(file_names, file_dfs)}
    attachments = [attach_file(inner_dict['name'], inner_dict['df']) for idx, inner_dict in attach_dict.items()]

    # Anexando arquivos
    for name, content in attachments or []:
        file = FileAttachment(name=name, content=content)
        m.attach(file)

    # Enviando mensagem
    m.send_and_save()

# Definindo função capaz de encapsular a mensagem
def send_mail_one_file(username, password, server, mail_box, subject, mail_body, mail_to, mail_signature='',
                       df=None, df_on_body=False, df_on_attachment=False, attachment_filename='file.csv'):
    
    # Configurando conta de envio
    account = connect_exchange(username=username, password=password, server=server, mail_box=mail_box)

    # Verificando dados a serem enviados no corpo
    if df_on_body and df is not None:
        html_body = format_mail_body(mail_body, df=df, mail_signature=mail_signature)
    else:
        html_body = format_mail_body(mail_body, mail_signature=mail_signature)

    # Preparando mensagem
    m = Message(account=account,
                subject=subject,
                body=html_body,
                to_recipients=mail_to)
    
    # Verificando anexos e preparando estruturas
    if df_on_attachment and df is not None:
        attachments = [attach_file(name=attachment_filename, df=df)]

        # Anexando arquivos
        for name, content in attachments or []:
            file = FileAttachment(name=name, content=content)
            m.attach(file)

    # Enviando mensagem
    m.send_and_save()
    

In [3]:
# Lendo variáveis de ambiente
load_dotenv()

# Definindo variáveis de configuração do e-mail
USERNAME = 'thipanini94@gmail.com'
PWD = os.getenv('PWD')
SERVER = 'outlook.office365.com'
MAIL_BOX = 'thipanini94@gmail.com'
MAIL_TO = ['thipanini94@gmail.com']
SUBJECT = 'Encapsulamento de E-mail'

# Definindo variáveis de formatação do e-mail
MAIL_BODY = "Relatório enviado com sucesso! Dados:<br/><br/>"
MAIL_SIGNATURE = '<br>Att,<br>Thiago'
DF_ON_BODY = True

# Variáveis testes de anexo de arquivos
CSV_FILEPATH = r'D:\Users\thiagoPanini\data-science-repos\kaggle\01_german_credit_risk\german_credit_data.csv'
PNG_FILEPATH = r'D:\Users\thiagoPanini\data-science-repos\kaggle\01_german_credit_risk\confusion_matrix_final.png'
TXT_FILEPATH = r'D:\Users\thiagoPanini\data-science-repos\kaggle\04_brazilian_e-commerce\dataset_test.txt'
XLSX_FILEPATH = r'C:\Users\thipa\Desktop\certificacao_alteryx\core\praticas\4701229_3BJP240W.xlsx'

FLAG_ATTACHMENT = True
FILE_PATHS = [CSV_FILEPATH, XLSX_FILEPATH, TXT_FILEPATH]

In [4]:
# Lendo bases de dados
dfs = []
for path in FILE_PATHS:
    try:
        dfs.append(pd.read_csv(path))
    except:
        dfs.append(pd.read_excel(path))

# Preparando listas
inputs = [i for i in range(1, 4)]
names = ['german.csv', 'ayx_ore.xlsx', 'brazilian.txt']
flag_body = [0, 0, 0]
flag_attach = [0, 0, 0]

# Criando DataFrame vazio
meta_df = pd.DataFrame({})
meta_df['input'] = inputs
meta_df['name'] = names
meta_df['df'] = dfs
meta_df['flag_body'] = flag_body
meta_df['flag_attach'] = flag_attach
meta_df

,input,name,df,flag_body,flag_attach
0,1,german.csv,Unnamed: 0 Age Sex Job Housing Savi...,0,0
1,2,ayx_ore.xlsx,emails Entries 0 ...,0,0
2,3,brazilian.txt,produt bom cheg ...,0,0


In [5]:
# Enviando email
send_mail_mult_files(meta_df=meta_df,
                     username=USERNAME,
                     password=PWD,
                     server=SERVER,
                     mail_box=MAIL_BOX,
                     subject=SUBJECT,
                     mail_body=MAIL_BODY,
                     mail_signature=MAIL_SIGNATURE,
                     mail_to=MAIL_TO)

In [67]:
# Enviando email
send_mail_one_file(username=USERNAME,
                   password=PWD,
                   server=SERVER,
                   mail_box=MAIL_BOX,
                   subject=SUBJECT,
                   mail_body=MAIL_BODY,
                   mail_signature=MAIL_SIGNATURE,
                   mail_to=MAIL_TO,
                   df=dfs[1],
                   df_on_body=False,
                   df_on_attachment=True,
                   attachment_filename='ayx_core.xlsx')

In [59]:
# Corpo de e-mail 
MAIL_BODY = """
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional //EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:v="urn:schemas-microsoft-com:vml">
<head>
<!--[if gte mso 9]><xml><o:OfficeDocumentSettings><o:AllowPNG/><o:PixelsPerInch>96</o:PixelsPerInch></o:OfficeDocumentSettings></xml><![endif]-->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width" name="viewport"/>
<!--[if !mso]><!-->
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<!--<![endif]-->
<title></title>
<!--[if !mso]><!-->
<link href="https://fonts.googleapis.com/css?family=Montserrat" rel="stylesheet" type="text/css"/>
<!--<![endif]-->
<style type="text/css">
		body {
			margin: 0;
			padding: 0;
		}

		table,
		td,
		tr {
			vertical-align: top;
			border-collapse: collapse;
		}

		* {
			line-height: inherit;
		}

		a[x-apple-data-detectors=true] {
			color: inherit !important;
			text-decoration: none !important;
		}
	</style>
<style id="media-query" type="text/css">
		@media (max-width: 660px) {

			.block-grid,
			.col {
				min-width: 320px !important;
				max-width: 100% !important;
				display: block !important;
			}

			.block-grid {
				width: 100% !important;
			}

			.col {
				width: 100% !important;
			}

			.col_cont {
				margin: 0 auto;
			}

			img.fullwidth,
			img.fullwidthOnMobile {
				max-width: 100% !important;
			}

			.no-stack .col {
				min-width: 0 !important;
				display: table-cell !important;
			}

			.no-stack.two-up .col {
				width: 50% !important;
			}

			.no-stack .col.num2 {
				width: 16.6% !important;
			}

			.no-stack .col.num3 {
				width: 25% !important;
			}

			.no-stack .col.num4 {
				width: 33% !important;
			}

			.no-stack .col.num5 {
				width: 41.6% !important;
			}

			.no-stack .col.num6 {
				width: 50% !important;
			}

			.no-stack .col.num7 {
				width: 58.3% !important;
			}

			.no-stack .col.num8 {
				width: 66.6% !important;
			}

			.no-stack .col.num9 {
				width: 75% !important;
			}

			.no-stack .col.num10 {
				width: 83.3% !important;
			}

			.video-block {
				max-width: none !important;
			}

			.mobile_hide {
				min-height: 0px;
				max-height: 0px;
				max-width: 0px;
				display: none;
				overflow: hidden;
				font-size: 0px;
			}

			.desktop_hide {
				display: block !important;
				max-height: none !important;
			}
		}
	</style>
<style id="icon-media-query" type="text/css">
		@media (max-width: 660px) {
			.icons-inner {
				text-align: center;
			}

			.icons-inner td {
				margin: 0 auto;
			}
		}
	</style>
</head>
<body class="clean-body" style="margin: 0; padding: 0; -webkit-text-size-adjust: 100%; background-color: #070918;">
<!--[if IE]><div class="ie-browser"><![endif]-->
<table bgcolor="#070918" cellpadding="0" cellspacing="0" class="nl-container" role="presentation" style="table-layout: fixed; vertical-align: top; min-width: 320px; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; background-color: #070918; width: 100%;" valign="top" width="100%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td style="word-break: break-word; vertical-align: top;" valign="top">
<!--[if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td align="center" style="background-color:#070918"><![endif]-->
<div style="background-color:transparent;">
<div class="block-grid" style="min-width: 320px; max-width: 640px; overflow-wrap: break-word; word-wrap: break-word; word-break: break-word; Margin: 0 auto; background-color: #000000;">
<div style="border-collapse: collapse;display: table;width: 100%;background-color:#000000;background-image:url('images/e2fe2c96-f4b8-4c68-bc87-16510ba2b053.jpg');background-position:top left;background-repeat:no-repeat">
<!--[if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="background-color:transparent;"><tr><td align="center"><table cellpadding="0" cellspacing="0" border="0" style="width:640px"><tr class="layout-full-width" style="background-color:#000000"><![endif]-->
<!--[if (mso)|(IE)]><td align="center" width="640" style="background-color:#000000;width:640px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top:0px; padding-bottom:0px;"><![endif]-->
<div class="col num12" style="min-width: 320px; max-width: 640px; display: table-cell; vertical-align: top; width: 640px;">
<div class="col_cont" style="width:100% !important;">
<!--[if (!mso)&(!IE)]><!-->
<div style="border-top:0px solid transparent; border-left:0px solid transparent; border-bottom:0px solid transparent; border-right:0px solid transparent; padding-top:0px; padding-bottom:0px; padding-right: 0px; padding-left: 0px;">
<!--<![endif]-->
<table cellpadding="0" cellspacing="0" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt;" valign="top" width="100%">
<tr style="vertical-align: top;" valign="top">
<td align="center" style="word-break: break-word; vertical-align: top; padding-bottom: 20px; padding-left: 20px; padding-right: 20px; padding-top: 20px; text-align: center; width: 100%;" valign="top" width="100%">
<h1 style="color:#ffffff;direction:ltr;font-family:Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif;font-size:23px;font-weight:normal;letter-spacing:4px;line-height:120%;text-align:center;margin-top:0;margin-bottom:0;">Ciência de Decisão Tema e Subtema</h1>
</td>
</tr>
</table>
<table border="0" cellpadding="0" cellspacing="0" class="divider" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top" width="100%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td class="divider_inner" style="word-break: break-word; vertical-align: top; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; padding-top: 0px; padding-right: 0px; padding-bottom: 0px; padding-left: 0px;" valign="top">
<table align="center" border="0" cellpadding="0" cellspacing="0" class="divider_content" height="50" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; border-top: 0px solid transparent; height: 50px; width: 100%;" valign="top" width="100%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td height="50" style="word-break: break-word; vertical-align: top; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top"><span></span></td>
</tr>
</tbody>
</table>
</td>
</tr>
</tbody>
</table>
<table cellpadding="0" cellspacing="0" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt;" valign="top" width="100%">
<tr style="vertical-align: top;" valign="top">
<td align="center" style="word-break: break-word; vertical-align: top; padding-bottom: 0px; padding-left: 0px; padding-right: 0px; padding-top: 0px; text-align: center; width: 100%;" valign="top" width="100%">
<h1 style="color:#ffffff;direction:ltr;font-family:Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif;font-size:60px;font-weight:normal;letter-spacing:6px;line-height:120%;text-align:center;margin-top:0;margin-bottom:0;"><strong>Projeto X</strong></h1>
</td>
</tr>
</table>
<div align="center" class="img-container center autowidth" style="padding-right: 0px;padding-left: 0px;">
<!--[if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr style="line-height:0px"><td style="padding-right: 0px;padding-left: 0px;" align="center"><![endif]--><img align="center" border="0" class="center autowidth" src="images/2021_1.png" style="text-decoration: none; -ms-interpolation-mode: bicubic; height: auto; border: 0; width: 100%; max-width: 536px; display: block;" width="536"/>
<!--[if mso]></td></tr></table><![endif]-->
</div>
<table border="0" cellpadding="0" cellspacing="0" class="divider" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top" width="100%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td class="divider_inner" style="word-break: break-word; vertical-align: top; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; padding-top: 0px; padding-right: 0px; padding-bottom: 0px; padding-left: 0px;" valign="top">
<table align="center" border="0" cellpadding="0" cellspacing="0" class="divider_content" height="65" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; border-top: 0px solid transparent; height: 65px; width: 100%;" valign="top" width="100%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td height="65" style="word-break: break-word; vertical-align: top; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top"><span></span></td>
</tr>
</tbody>
</table>
</td>
</tr>
</tbody>
</table>
<div align="center" class="button-container" style="padding-top:10px;padding-right:10px;padding-bottom:10px;padding-left:10px;">
<!--[if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="border-spacing: 0; border-collapse: collapse; mso-table-lspace:0pt; mso-table-rspace:0pt;"><tr><td style="padding-top: 10px; padding-right: 10px; padding-bottom: 10px; padding-left: 10px" align="center"><v:roundrect xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="urn:schemas-microsoft-com:office:word" href="https://www.example.com" style="height:33pt;width:246.75pt;v-text-anchor:middle;" arcsize="55%" strokeweight="0.75pt" strokecolor="#FFFFFF" fill="false"><w:anchorlock/><v:textbox inset="0,0,0,0"><center style="color:#ffffff; font-family:Tahoma, sans-serif; font-size:16px"><![endif]--><a href="https://www.example.com" style="-webkit-text-size-adjust: none; text-decoration: none; display: inline-block; color: #ffffff; background-color: transparent; border-radius: 24px; -webkit-border-radius: 24px; -moz-border-radius: 24px; width: auto; width: auto; border-top: 1px solid #FFFFFF; border-right: 1px solid #FFFFFF; border-bottom: 1px solid #FFFFFF; border-left: 1px solid #FFFFFF; padding-top: 5px; padding-bottom: 5px; font-family: Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif; text-align: center; mso-border-alt: none; word-break: keep-all;" target="_blank"><span style="padding-left:30px;padding-right:30px;font-size:16px;display:inline-block;letter-spacing:3px;"><span style="font-size: 16px; line-height: 2; word-break: break-word; mso-line-height-alt: 32px;"><strong>ACESSO AO PAINEL</strong></span></span></a>
<!--[if mso]></center></v:textbox></v:roundrect></td></tr></table><![endif]-->
</div>
<div align="center" class="img-container center autowidth" style="padding-right: 0px;padding-left: 0px;">
<!--[if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr style="line-height:0px"><td style="padding-right: 0px;padding-left: 0px;" align="center"><![endif]--><img align="center" border="0" class="center autowidth" src="images/mountain-foreground.png" style="text-decoration: none; -ms-interpolation-mode: bicubic; height: auto; border: 0; width: 100%; max-width: 640px; display: block;" width="640"/>
<!--[if mso]></td></tr></table><![endif]-->
</div>
<table border="0" cellpadding="0" cellspacing="0" class="divider" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top" width="100%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td class="divider_inner" style="word-break: break-word; vertical-align: top; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; padding-top: 0px; padding-right: 0px; padding-bottom: 0px; padding-left: 0px;" valign="top">
<table align="center" border="0" cellpadding="0" cellspacing="0" class="divider_content" height="10" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; border-top: 0px solid transparent; height: 10px; width: 100%;" valign="top" width="100%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td height="10" style="word-break: break-word; vertical-align: top; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top"><span></span></td>
</tr>
</tbody>
</table>
</td>
</tr>
</tbody>
</table>
<!--[if (!mso)&(!IE)]><!-->
</div>
<!--<![endif]-->
</div>
</div>
<!--[if (mso)|(IE)]></td></tr></table><![endif]-->
<!--[if (mso)|(IE)]></td></tr></table></td></tr></table><![endif]-->
</div>
</div>
</div>
<div style="background-color:transparent;">
<div class="block-grid mixed-two-up" style="min-width: 320px; max-width: 640px; overflow-wrap: break-word; word-wrap: break-word; word-break: break-word; Margin: 0 auto; background-color: #000000;">
<div style="border-collapse: collapse;display: table;width: 100%;background-color:#000000;">
<!--[if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="background-color:transparent;"><tr><td align="center"><table cellpadding="0" cellspacing="0" border="0" style="width:640px"><tr class="layout-full-width" style="background-color:#000000"><![endif]-->
<!--[if (mso)|(IE)]><td align="center" width="266" style="background-color:#000000;width:266px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 5px; padding-left: 25px; padding-top:30px; padding-bottom:30px;"><![endif]-->
<div class="col num5" style="display: table-cell; vertical-align: top; max-width: 320px; min-width: 265px; width: 266px;">
<div class="col_cont" style="width:100% !important;">
<!--[if (!mso)&(!IE)]><!-->
<div style="border-top:0px solid transparent; border-left:0px solid transparent; border-bottom:0px solid transparent; border-right:0px solid transparent; padding-top:30px; padding-bottom:30px; padding-right: 5px; padding-left: 25px;">
<!--<![endif]-->
<div align="center" class="img-container center autowidth" style="padding-right: 0px;padding-left: 0px;">
<!--[if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr style="line-height:0px"><td style="padding-right: 0px;padding-left: 0px;" align="center"><![endif]--><img align="center" alt="Lifestyle photo of food" border="0" class="center autowidth" src="images/earthspin-compressed.gif" style="text-decoration: none; -ms-interpolation-mode: bicubic; height: auto; border: 0; width: 100%; max-width: 236px; display: block;" title="Lifestyle photo of food" width="236"/>
<!--[if mso]></td></tr></table><![endif]-->
</div>
<!--[if (!mso)&(!IE)]><!-->
</div>
<!--<![endif]-->
</div>
</div>
<!--[if (mso)|(IE)]></td></tr></table><![endif]-->
<!--[if (mso)|(IE)]></td><td align="center" width="373" style="background-color:#000000;width:373px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 20px; padding-left: 20px; padding-top:30px; padding-bottom:30px;"><![endif]-->
<div class="col num7" style="display: table-cell; vertical-align: top; max-width: 320px; min-width: 371px; width: 373px;">
<div class="col_cont" style="width:100% !important;">
<!--[if (!mso)&(!IE)]><!-->
<div style="border-top:0px solid transparent; border-left:0px solid transparent; border-bottom:0px solid transparent; border-right:0px solid transparent; padding-top:30px; padding-bottom:30px; padding-right: 20px; padding-left: 20px;">
<!--<![endif]-->
<table cellpadding="0" cellspacing="0" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt;" valign="top" width="100%">
<tr style="vertical-align: top;" valign="top">
<td align="center" style="word-break: break-word; vertical-align: top; padding-bottom: 0px; padding-left: 0px; padding-right: 0px; padding-top: 0px; text-align: center; width: 100%;" valign="top" width="100%">
<h3 style="color:#a4b1cc;direction:ltr;font-family:Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif;font-size:16px;font-weight:normal;letter-spacing:2px;line-height:120%;text-align:left;margin-top:0;margin-bottom:0;">Produto 01</h3>
</td>
</tr>
</table>
<table cellpadding="0" cellspacing="0" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt;" valign="top" width="100%">
<tr style="vertical-align: top;" valign="top">
<td align="center" style="word-break: break-word; vertical-align: top; padding-bottom: 0px; padding-left: 0px; padding-right: 0px; padding-top: 0px; text-align: center; width: 100%;" valign="top" width="100%">
<h3 style="color:#a4b1cc;direction:ltr;font-family:Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif;font-size:22px;font-weight:normal;letter-spacing:6px;line-height:120%;text-align:left;margin-top:0;margin-bottom:0;"><strong>Indicador Dia</strong><br/></h3>
</td>
</tr>
</table>
<table border="0" cellpadding="0" cellspacing="0" class="divider" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top" width="100%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td class="divider_inner" style="word-break: break-word; vertical-align: top; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; padding-top: 10px; padding-right: 10px; padding-bottom: 10px; padding-left: 10px;" valign="top">
<table align="center" border="0" cellpadding="0" cellspacing="0" class="divider_content" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; border-top: 1px solid #BBBBBB; width: 100%;" valign="top" width="100%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td style="word-break: break-word; vertical-align: top; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top"><span></span></td>
</tr>
</tbody>
</table>
</td>
</tr>
</tbody>
</table>
<table cellpadding="0" cellspacing="0" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt;" valign="top" width="100%">
<tr style="vertical-align: top;" valign="top">
<td align="center" style="word-break: break-word; vertical-align: top; padding-bottom: 20px; padding-left: 20px; padding-right: 20px; padding-top: 20px; text-align: center; width: 100%;" valign="top" width="100%">
<h1 style="color:#ffffff;direction:ltr;font-family:Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif;font-size:60px;font-weight:normal;letter-spacing:6px;line-height:120%;text-align:center;margin-top:0;margin-bottom:0;"><strong>ID01</strong></h1>
</td>
</tr>
</table>
<table cellpadding="0" cellspacing="0" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt;" valign="top" width="100%">
<tr style="vertical-align: top;" valign="top">
<td align="center" style="word-break: break-word; vertical-align: top; padding-bottom: 0px; padding-left: 0px; padding-right: 0px; padding-top: 0px; text-align: center; width: 100%;" valign="top" width="100%">
<h3 style="color:#a4b1cc;direction:ltr;font-family:Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif;font-size:16px;font-weight:normal;letter-spacing:2px;line-height:120%;text-align:center;margin-top:0;margin-bottom:0;">desc_id01</h3>
</td>
</tr>
</table>
<table border="0" cellpadding="0" cellspacing="0" class="divider" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top" width="100%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td class="divider_inner" style="word-break: break-word; vertical-align: top; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; padding-top: 10px; padding-right: 10px; padding-bottom: 10px; padding-left: 10px;" valign="top">
<table align="center" border="0" cellpadding="0" cellspacing="0" class="divider_content" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; border-top: 1px solid #BBBBBB; width: 100%;" valign="top" width="100%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td style="word-break: break-word; vertical-align: top; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top"><span></span></td>
</tr>
</tbody>
</table>
</td>
</tr>
</tbody>
</table>
<div align="left" class="button-container" style="padding-top:10px;padding-right:0px;padding-bottom:10px;padding-left:0px;">
<!--[if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="border-spacing: 0; border-collapse: collapse; mso-table-lspace:0pt; mso-table-rspace:0pt;"><tr><td style="padding-top: 10px; padding-right: 0px; padding-bottom: 10px; padding-left: 0px" align="left"><v:roundrect xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="urn:schemas-microsoft-com:office:word" href="https://www.example.com" style="height:30pt;width:204pt;v-text-anchor:middle;" arcsize="60%" strokeweight="0.75pt" strokecolor="#A4B1CC" fill="false"><w:anchorlock/><v:textbox inset="0,0,0,0"><center style="color:#a4b1cc; font-family:Tahoma, sans-serif; font-size:14px"><![endif]--><a href="https://www.example.com" style="-webkit-text-size-adjust: none; text-decoration: none; display: inline-block; color: #a4b1cc; background-color: transparent; border-radius: 24px; -webkit-border-radius: 24px; -moz-border-radius: 24px; width: auto; width: auto; border-top: 1px solid #A4B1CC; border-right: 1px solid #A4B1CC; border-bottom: 1px solid #A4B1CC; border-left: 1px solid #A4B1CC; padding-top: 5px; padding-bottom: 5px; font-family: Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif; text-align: center; mso-border-alt: none; word-break: keep-all;" target="_blank"><span style="padding-left:25px;padding-right:25px;font-size:14px;display:inline-block;letter-spacing:2px;"><span style="line-height: 32px; word-break: break-word;"><span data-mce-style="font-size: 14px; line-height: 28px;" style="font-size: 14px; line-height: 28px;"><strong>ADD TO </strong></span><span data-mce-style="font-size: 14px; line-height: 28px;" style="font-size: 14px; line-height: 28px;"><strong>CALENDAR</strong></span></span></span></a>
<!--[if mso]></center></v:textbox></v:roundrect></td></tr></table><![endif]-->
</div>
<!--[if (!mso)&(!IE)]><!-->
</div>
<!--<![endif]-->
</div>
</div>
<!--[if (mso)|(IE)]></td></tr></table><![endif]-->
<!--[if (mso)|(IE)]></td></tr></table></td></tr></table><![endif]-->
</div>
</div>
</div>
<div style="background-color:transparent;">
<div class="block-grid" style="min-width: 320px; max-width: 640px; overflow-wrap: break-word; word-wrap: break-word; word-break: break-word; Margin: 0 auto; background-color: #000000;">
<div style="border-collapse: collapse;display: table;width: 100%;background-color:#000000;">
<!--[if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="background-color:transparent;"><tr><td align="center"><table cellpadding="0" cellspacing="0" border="0" style="width:640px"><tr class="layout-full-width" style="background-color:#000000"><![endif]-->
<!--[if (mso)|(IE)]><td align="center" width="640" style="background-color:#000000;width:640px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top:0px; padding-bottom:0px;"><![endif]-->
<div class="col num12" style="min-width: 320px; max-width: 640px; display: table-cell; vertical-align: top; width: 640px;">
<div class="col_cont" style="width:100% !important;">
<!--[if (!mso)&(!IE)]><!-->
<div style="border-top:0px solid transparent; border-left:0px solid transparent; border-bottom:0px solid transparent; border-right:0px solid transparent; padding-top:0px; padding-bottom:0px; padding-right: 0px; padding-left: 0px;">
<!--<![endif]-->
<table border="0" cellpadding="0" cellspacing="0" class="divider" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top" width="100%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td class="divider_inner" style="word-break: break-word; vertical-align: top; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; padding-top: 0px; padding-right: 0px; padding-bottom: 0px; padding-left: 0px;" valign="top">
<table align="center" border="0" cellpadding="0" cellspacing="0" class="divider_content" height="44" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; border-top: 0px solid transparent; height: 44px; width: 100%;" valign="top" width="100%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td height="44" style="word-break: break-word; vertical-align: top; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top"><span></span></td>
</tr>
</tbody>
</table>
</td>
</tr>
</tbody>
</table>
<!--[if (!mso)&(!IE)]><!-->
</div>
<!--<![endif]-->
</div>
</div>
<!--[if (mso)|(IE)]></td></tr></table><![endif]-->
<!--[if (mso)|(IE)]></td></tr></table></td></tr></table><![endif]-->
</div>
</div>
</div>
<div style="background-color:transparent;">
<div class="block-grid mixed-two-up" style="min-width: 320px; max-width: 640px; overflow-wrap: break-word; word-wrap: break-word; word-break: break-word; Margin: 0 auto; background-color: #000000;">
<div style="border-collapse: collapse;display: table;width: 100%;background-color:#000000;;direction:rtl">
<!--[if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="background-color:transparent;"><tr><td align="center"><table cellpadding="0" cellspacing="0" border="0" style="width:640px"><tr class="layout-full-width" style="background-color:#000000"><![endif]-->
<!--[if (mso)|(IE)]><td align="center" width="266" style="background-color:#000000;width:266px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 20px; padding-left: 20px; padding-top:30px; padding-bottom:30px;"><![endif]-->
<div class="col num5" style="display: table-cell; vertical-align: top; max-width: 320px; min-width: 265px; width: 266px; direction: ltr;">
<div class="col_cont" style="width:100% !important;">
<!--[if (!mso)&(!IE)]><!-->
<div style="border-top:0px solid transparent; border-left:0px solid transparent; border-bottom:0px solid transparent; border-right:0px solid transparent; padding-top:30px; padding-bottom:30px; padding-right: 20px; padding-left: 20px;">
<!--<![endif]-->
<div align="center" class="img-container center autowidth" style="padding-right: 0px;padding-left: 0px;">
<!--[if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr style="line-height:0px"><td style="padding-right: 0px;padding-left: 0px;" align="center"><![endif]--><img align="center" alt="Lifestyle photo of food" border="0" class="center autowidth" src="images/ezgif-6-8fa31ed13bb9b.gif" style="text-decoration: none; -ms-interpolation-mode: bicubic; height: auto; border: 0; width: 100%; max-width: 226px; display: block;" title="Lifestyle photo of food" width="226"/>
<!--[if mso]></td></tr></table><![endif]-->
</div>
<!--[if (!mso)&(!IE)]><!-->
</div>
<!--<![endif]-->
</div>
</div>
<!--[if (mso)|(IE)]></td></tr></table><![endif]-->
<!--[if (mso)|(IE)]></td><td align="center" width="373" style="background-color:#000000;width:373px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 5px; padding-left: 25px; padding-top:30px; padding-bottom:30px;"><![endif]-->
<div class="col num7" style="display: table-cell; vertical-align: top; max-width: 320px; min-width: 371px; width: 373px; direction: ltr;">
<div class="col_cont" style="width:100% !important;">
<!--[if (!mso)&(!IE)]><!-->
<div style="border-top:0px solid transparent; border-left:0px solid transparent; border-bottom:0px solid transparent; border-right:0px solid transparent; padding-top:30px; padding-bottom:30px; padding-right: 5px; padding-left: 25px;">
<!--<![endif]-->
<table cellpadding="0" cellspacing="0" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt;" valign="top" width="100%">
<tr style="vertical-align: top;" valign="top">
<td align="center" style="word-break: break-word; vertical-align: top; padding-bottom: 0px; padding-left: 0px; padding-right: 0px; padding-top: 0px; text-align: center; width: 100%;" valign="top" width="100%">
<h3 style="color:#a4b1cc;direction:ltr;font-family:Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif;font-size:16px;font-weight:normal;letter-spacing:3px;line-height:120%;text-align:left;margin-top:0;margin-bottom:0;">7:30pm - 8:30pm</h3>
</td>
</tr>
</table>
<table cellpadding="0" cellspacing="0" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt;" valign="top" width="100%">
<tr style="vertical-align: top;" valign="top">
<td align="center" style="word-break: break-word; vertical-align: top; padding-bottom: 0px; padding-left: 0px; padding-right: 0px; padding-top: 0px; text-align: center; width: 100%;" valign="top" width="100%">
<h3 style="color:#a4b1cc;direction:ltr;font-family:Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif;font-size:22px;font-weight:normal;letter-spacing:6px;line-height:120%;text-align:left;margin-top:0;margin-bottom:0;"><strong>LIGHTS OUT</strong><br/></h3>
</td>
</tr>
</table>
<!--[if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top: 15px; padding-bottom: 15px; font-family: Tahoma, sans-serif"><![endif]-->
<div style="color:#a4b1cc;font-family:Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif;line-height:1.5;padding-top:15px;padding-right:0px;padding-bottom:15px;padding-left:0px;">
<div class="txtTinyMce-wrapper" style="line-height: 1.5; font-size: 12px; color: #a4b1cc; font-family: Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif; mso-line-height-alt: 18px;">
<p style="font-size: 14px; line-height: 1.5; word-break: break-word; text-align: left; mso-line-height-alt: 21px; margin: 0;">Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pulvinar orci, duis auctor faucibus. Dui nunc ut aliquam odio. Risus tempus mollis nunc non magnis tortor commodo quis.</p>
</div>
</div>
<!--[if mso]></td></tr></table><![endif]-->
<div align="left" class="button-container" style="padding-top:10px;padding-right:0px;padding-bottom:10px;padding-left:0px;">
<!--[if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="border-spacing: 0; border-collapse: collapse; mso-table-lspace:0pt; mso-table-rspace:0pt;"><tr><td style="padding-top: 10px; padding-right: 0px; padding-bottom: 10px; padding-left: 0px" align="left"><v:roundrect xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="urn:schemas-microsoft-com:office:word" href="https://www.example.com" style="height:30pt;width:207pt;v-text-anchor:middle;" arcsize="60%" strokeweight="0.75pt" strokecolor="#A4B1CC" fill="false"><w:anchorlock/><v:textbox inset="0,0,0,0"><center style="color:#a4b1cc; font-family:Tahoma, sans-serif; font-size:14px"><![endif]--><a href="https://www.example.com" style="-webkit-text-size-adjust: none; text-decoration: none; display: inline-block; color: #a4b1cc; background-color: transparent; border-radius: 24px; -webkit-border-radius: 24px; -moz-border-radius: 24px; width: auto; width: auto; border-top: 1px solid #A4B1CC; border-right: 1px solid #A4B1CC; border-bottom: 1px solid #A4B1CC; border-left: 1px solid #A4B1CC; padding-top: 5px; padding-bottom: 5px; font-family: Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif; text-align: center; mso-border-alt: none; word-break: keep-all;" target="_blank"><span style="padding-left:25px;padding-right:25px;font-size:14px;display:inline-block;letter-spacing:2px;"><span style="line-height: 32px; word-break: break-word;"><strong><span data-mce-style="font-size: 14px; line-height: 28px;" style="font-size: 14px; line-height: 28px;">SPREAD THE WORD</span></strong></span></span></a>
<!--[if mso]></center></v:textbox></v:roundrect></td></tr></table><![endif]-->
</div>
<!--[if (!mso)&(!IE)]><!-->
</div>
<!--<![endif]-->
</div>
</div>
<!--[if (mso)|(IE)]></td></tr></table><![endif]-->
<!--[if (mso)|(IE)]></td></tr></table></td></tr></table><![endif]-->
</div>
</div>
</div>
<div style="background-color:transparent;">
<div class="block-grid" style="min-width: 320px; max-width: 640px; overflow-wrap: break-word; word-wrap: break-word; word-break: break-word; Margin: 0 auto; background-color: #000000;">
<div style="border-collapse: collapse;display: table;width: 100%;background-color:#000000;">
<!--[if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="background-color:transparent;"><tr><td align="center"><table cellpadding="0" cellspacing="0" border="0" style="width:640px"><tr class="layout-full-width" style="background-color:#000000"><![endif]-->
<!--[if (mso)|(IE)]><td align="center" width="640" style="background-color:#000000;width:640px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top:15px; padding-bottom:15px;"><![endif]-->
<div class="col num12" style="min-width: 320px; max-width: 640px; display: table-cell; vertical-align: top; width: 640px;">
<div class="col_cont" style="width:100% !important;">
<!--[if (!mso)&(!IE)]><!-->
<div style="border-top:0px solid transparent; border-left:0px solid transparent; border-bottom:0px solid transparent; border-right:0px solid transparent; padding-top:15px; padding-bottom:15px; padding-right: 0px; padding-left: 0px;">
<!--<![endif]-->
<table border="0" cellpadding="0" cellspacing="0" class="divider" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top" width="100%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td class="divider_inner" style="word-break: break-word; vertical-align: top; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; padding-top: 10px; padding-right: 10px; padding-bottom: 10px; padding-left: 10px;" valign="top">
<table align="center" border="0" cellpadding="0" cellspacing="0" class="divider_content" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; border-top: 1px solid #3F465C; width: 85%;" valign="top" width="85%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td style="word-break: break-word; vertical-align: top; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top"><span></span></td>
</tr>
</tbody>
</table>
</td>
</tr>
</tbody>
</table>
<table cellpadding="0" cellspacing="0" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt;" valign="top" width="100%">
<tr style="vertical-align: top;" valign="top">
<td align="center" style="word-break: break-word; vertical-align: top; padding-bottom: 20px; padding-left: 10px; padding-right: 10px; padding-top: 20px; text-align: center; width: 100%;" valign="top" width="100%">
<h2 style="color:#a4b1cc;direction:ltr;font-family:Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif;font-size:19px;font-weight:normal;letter-spacing:6px;line-height:120%;text-align:center;margin-top:0;margin-bottom:0;"><strong>BE A PART OF THE CHANGE</strong></h2>
</td>
</tr>
</table>
<div align="center" class="button-container" style="padding-top:35px;padding-right:10px;padding-bottom:30px;padding-left:10px;">
<!--[if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="border-spacing: 0; border-collapse: collapse; mso-table-lspace:0pt; mso-table-rspace:0pt;"><tr><td style="padding-top: 35px; padding-right: 10px; padding-bottom: 30px; padding-left: 10px" align="center"><v:roundrect xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="urn:schemas-microsoft-com:office:word" href="https://www.example.com" style="height:33pt;width:249pt;v-text-anchor:middle;" arcsize="55%" strokeweight="0.75pt" strokecolor="#FFFFFF" fill="false"><w:anchorlock/><v:textbox inset="0,0,0,0"><center style="color:#ffffff; font-family:Tahoma, sans-serif; font-size:16px"><![endif]--><a href="https://www.example.com" style="-webkit-text-size-adjust: none; text-decoration: none; display: inline-block; color: #ffffff; background-color: transparent; border-radius: 24px; -webkit-border-radius: 24px; -moz-border-radius: 24px; width: auto; width: auto; border-top: 1px solid #FFFFFF; border-right: 1px solid #FFFFFF; border-bottom: 1px solid #FFFFFF; border-left: 1px solid #FFFFFF; padding-top: 5px; padding-bottom: 5px; font-family: Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif; text-align: center; mso-border-alt: none; word-break: keep-all;" target="_blank"><span style="padding-left:30px;padding-right:30px;font-size:16px;display:inline-block;letter-spacing:3px;"><span style="font-size: 16px; line-height: 2; word-break: break-word; mso-line-height-alt: 32px;"><strong>SPREAD THE WORD</strong></span></span></a>
<!--[if mso]></center></v:textbox></v:roundrect></td></tr></table><![endif]-->
</div>
<!--[if (!mso)&(!IE)]><!-->
</div>
<!--<![endif]-->
</div>
</div>
<!--[if (mso)|(IE)]></td></tr></table><![endif]-->
<!--[if (mso)|(IE)]></td></tr></table></td></tr></table><![endif]-->
</div>
</div>
</div>
<div style="background-color:transparent;">
<div class="block-grid" style="min-width: 320px; max-width: 640px; overflow-wrap: break-word; word-wrap: break-word; word-break: break-word; Margin: 0 auto; background-color: #000000;">
<div style="border-collapse: collapse;display: table;width: 100%;background-color:#000000;">
<!--[if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="background-color:transparent;"><tr><td align="center"><table cellpadding="0" cellspacing="0" border="0" style="width:640px"><tr class="layout-full-width" style="background-color:#000000"><![endif]-->
<!--[if (mso)|(IE)]><td align="center" width="640" style="background-color:#000000;width:640px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top:5px; padding-bottom:5px;"><![endif]-->
<div class="col num12" style="min-width: 320px; max-width: 640px; display: table-cell; vertical-align: top; width: 640px;">
<div class="col_cont" style="width:100% !important;">
<!--[if (!mso)&(!IE)]><!-->
<div style="border-top:0px solid transparent; border-left:0px solid transparent; border-bottom:0px solid transparent; border-right:0px solid transparent; padding-top:5px; padding-bottom:5px; padding-right: 0px; padding-left: 0px;">
<!--<![endif]-->
<table cellpadding="0" cellspacing="0" class="social_icons" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt;" valign="top" width="100%">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td style="word-break: break-word; vertical-align: top; padding-top: 10px; padding-right: 10px; padding-bottom: 10px; padding-left: 10px;" valign="top">
<table align="center" cellpadding="0" cellspacing="0" class="social_table" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-tspace: 0; mso-table-rspace: 0; mso-table-bspace: 0; mso-table-lspace: 0;" valign="top">
<tbody>
<tr align="center" style="vertical-align: top; display: inline-block; text-align: center;" valign="top">
<td style="word-break: break-word; vertical-align: top; padding-bottom: 0; padding-right: 7.5px; padding-left: 7.5px;" valign="top"><a href="https://www.facebook.com/" target="_blank"><img alt="Facebook" height="32" src="images/facebook2x.png" style="text-decoration: none; -ms-interpolation-mode: bicubic; height: auto; border: 0; display: block;" title="facebook" width="32"/></a></td>
<td style="word-break: break-word; vertical-align: top; padding-bottom: 0; padding-right: 7.5px; padding-left: 7.5px;" valign="top"><a href="https://www.twitter.com/" target="_blank"><img alt="Twitter" height="32" src="images/twitter2x.png" style="text-decoration: none; -ms-interpolation-mode: bicubic; height: auto; border: 0; display: block;" title="twitter" width="32"/></a></td>
<td style="word-break: break-word; vertical-align: top; padding-bottom: 0; padding-right: 7.5px; padding-left: 7.5px;" valign="top"><a href="https://www.linkedin.com/" target="_blank"><img alt="Linkedin" height="32" src="images/linkedin2x.png" style="text-decoration: none; -ms-interpolation-mode: bicubic; height: auto; border: 0; display: block;" title="linkedin" width="32"/></a></td>
<td style="word-break: break-word; vertical-align: top; padding-bottom: 0; padding-right: 7.5px; padding-left: 7.5px;" valign="top"><a href="https://www.instagram.com/" target="_blank"><img alt="Instagram" height="32" src="images/instagram2x.png" style="text-decoration: none; -ms-interpolation-mode: bicubic; height: auto; border: 0; display: block;" title="instagram" width="32"/></a></td>
</tr>
</tbody>
</table>
</td>
</tr>
</tbody>
</table>
<!--[if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 20px; padding-left: 20px; padding-top: 20px; padding-bottom: 40px; font-family: Tahoma, sans-serif"><![endif]-->
<div style="color:#a4b1cc;font-family:Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif;line-height:1.2;padding-top:20px;padding-right:20px;padding-bottom:40px;padding-left:20px;">
<div class="txtTinyMce-wrapper" style="color: #a4b1cc; font-size: 12px; line-height: 1.2; font-family: Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif; mso-line-height-alt: 14px;">
<p style="font-size: 12px; line-height: 1.2; text-align: center; word-break: break-word; mso-line-height-alt: 14px; margin: 0;"><span style="">Changed your mind? You can <span style="font-size: 12px;"><a href="http://www.example.com" rel="noopener" style="text-decoration: underline; color: #a4b1cc;" target="_blank">unsubscribe</a></span> at any time.</span></p>
</div>
</div>
<!--[if mso]></td></tr></table><![endif]-->
<!--[if (!mso)&(!IE)]><!-->
</div>
<!--<![endif]-->
</div>
</div>
<!--[if (mso)|(IE)]></td></tr></table><![endif]-->
<!--[if (mso)|(IE)]></td></tr></table></td></tr></table><![endif]-->
</div>
</div>
</div>
<div style="background-color:transparent;">
<div class="block-grid" style="min-width: 320px; max-width: 640px; overflow-wrap: break-word; word-wrap: break-word; word-break: break-word; Margin: 0 auto; background-color: transparent;">
<div style="border-collapse: collapse;display: table;width: 100%;background-color:transparent;">
<!--[if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="background-color:transparent;"><tr><td align="center"><table cellpadding="0" cellspacing="0" border="0" style="width:640px"><tr class="layout-full-width" style="background-color:transparent"><![endif]-->
<!--[if (mso)|(IE)]><td align="center" width="640" style="background-color:transparent;width:640px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top:5px; padding-bottom:5px;"><![endif]-->
<div class="col num12" style="min-width: 320px; max-width: 640px; display: table-cell; vertical-align: top; width: 640px;">
<div class="col_cont" style="width:100% !important;">
<!--[if (!mso)&(!IE)]><!-->
<div style="border-top:0px solid transparent; border-left:0px solid transparent; border-bottom:0px solid transparent; border-right:0px solid transparent; padding-top:5px; padding-bottom:5px; padding-right: 0px; padding-left: 0px;">
<!--<![endif]-->
<table cellpadding="0" cellspacing="0" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt;" valign="top" width="100%">
<tr style="vertical-align: top;" valign="top">
<td align="center" style="word-break: break-word; vertical-align: top; padding-top: 5px; padding-right: 0px; padding-bottom: 5px; padding-left: 0px; text-align: center;" valign="top">
<!--[if vml]><table align="left" cellpadding="0" cellspacing="0" role="presentation" style="display:inline-block;padding-left:0px;padding-right:0px;mso-table-lspace: 0pt;mso-table-rspace: 0pt;"><![endif]-->
<!--[if !vml]><!-->
<table cellpadding="0" cellspacing="0" class="icons-inner" role="presentation" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; display: inline-block; margin-right: -4px; padding-left: 0px; padding-right: 0px;" valign="top">
<!--<![endif]-->
<tr style="vertical-align: top;" valign="top">
<td align="center" style="word-break: break-word; vertical-align: top; text-align: center; padding-top: 5px; padding-bottom: 5px; padding-left: 5px; padding-right: 6px;" valign="top"><a href="https://www.designedwithbee.com/"><img align="center" alt="Designed with BEE" class="icon" height="32" src="images/bee.png" style="border:0;" width="null"/></a></td>
<td style="word-break: break-word; font-family: Montserrat, Trebuchet MS, Lucida Grande, Lucida Sans Unicode, Lucida Sans, Tahoma, sans-serif; font-size: 15px; color: #9d9d9d; vertical-align: middle; letter-spacing: undefined;" valign="middle"><a href="https://www.designedwithbee.com/" style="color:#9d9d9d;text-decoration:none;">Designed with BEE</a></td>
</tr>
</table>
</td>
</tr>
</table>
<!--[if (!mso)&(!IE)]><!-->
</div>
<!--<![endif]-->
</div>
</div>
<!--[if (mso)|(IE)]></td></tr></table><![endif]-->
<!--[if (mso)|(IE)]></td></tr></table></td></tr></table><![endif]-->
</div>
</div>
</div>
<!--[if (mso)|(IE)]></td></tr></table><![endif]-->
</td>
</tr>
</tbody>
</table>
<!--[if (IE)]></div><![endif]-->
</body>
</html>
"""

In [65]:
id01 = str(dfs[1]['Entries'].sum())
desc_id01 = 'Total de entradas encontradas'
MAIL_BODY = MAIL_BODY.replace('ID01', id01)
MAIL_BODY = MAIL_BODY.replace('desc_id01', desc_id01)